In [1]:
import os
import glob
import rasterio
import numpy as np
import csv

In [2]:
# Specify the directory containing your Sentinel-2 .tif files
root_folder = "/work/mech-ai-scratch/rtali/gis-sentinel2/final_s2_v3"  # Replace with your folder path

output_csv = 's2_band_statistics.csv'  # Name of the output CSV file

# Step 1: Identify the list of bands from the first subfolder
subfolders = [d for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]
if not subfolders:
    print("No subfolders found")
    exit()

first_subfolder = subfolders[0]
band_files = glob.glob(os.path.join(root_folder, first_subfolder, '4326_B*.tif'))
bands = [os.path.basename(f).split('_')[1].replace('.tif', '') for f in band_files]
print("Bands found:", bands)

# Step 2: Initialize a list to store statistics for each band
statistics = []

# Step 3: Process each band to compute overall statistics
for band in bands:
    print(f"\nProcessing band {band}")
    # Find all files for this band across all subfolders
    file_pattern = os.path.join(root_folder, '*', f'4326_{band}.tif')
    files = glob.glob(file_pattern)
    if not files:
        print(f"No files found for band {band}")
        continue
    
    # Initialize variables for statistics
    min_val = float('inf')         # Current minimum
    max_val = float('-inf')        # Current maximum
    sum_val = 0.0                  # Sum of pixel values
    sum_sq_val = 0.0               # Sum of squared pixel values
    count = 0                      # Total number of pixels
    
    # Step 4: Process each file for the current band
    for file in files:
        with rasterio.open(file) as src:
            data = src.read(1)     # Read the first (and assumed only) band
            pixels = data.flatten()  # Convert 2D array to 1D
            if pixels.size > 0:
                # Update statistics with this image's pixel values
                min_val = min(min_val, pixels.min())
                max_val = max(max_val, pixels.max())
                sum_val += np.sum(pixels, dtype=np.float64)
                sum_sq_val += np.sum(pixels.astype(np.float64) ** 2)
                count += pixels.size
    
    # Step 5: Compute the final statistics
    if count > 0:
        mean_val = sum_val / count
        variance = (sum_sq_val / count) - (mean_val ** 2)
        std_val = np.sqrt(variance)
        # Append the statistics for this band
        statistics.append({
            'Band': band,
            'Min': min_val,
            'Max': max_val,
            'Mean': mean_val,
            'StdDev': std_val
        })
    else:
        print(f"No pixels found for band {band}")

# Step 6: Save the statistics to a CSV file
if statistics:
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Band', 'Min', 'Max', 'Mean', 'StdDev']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for stat in statistics:
            writer.writerow(stat)
    print(f"Statistics saved to {output_csv}")
else:
    print("No statistics to save")